In [1]:
import tensorflow as tf

# Load the classification model
classification_model = tf.keras.models.load_model('TTC_classification_model.h5')

# Load the regression model
regression_model = tf.keras.models.load_model('TTC_regression_model.h5')

# Define the input shape for the classification model
classification_input_shape = classification_model.layers[0].input_shape

# Define the input layer for the regression model
regression_input_layer = tf.keras.layers.Input(shape=classification_input_shape[1:])

# Connect the output of the classification model to the input of the regression model
classification_output = classification_model(regression_input_layer)
regression_input = tf.keras.layers.concatenate([regression_input_layer, classification_output])

# Define a lambda layer that conditionally activates the regression model based on the output of the classification model
regression_activation = tf.keras.layers.Lambda(lambda x: x[0] * x[1], name='regression_activation')([regression_input, classification_output])

# Connect the regression model to the output of the classification model, using the regression_activation layer to conditionally activate it
regression_output = regression_activation * regression_model(regression_input)

# Define the combined model that includes both the classification and regression models
combined_model = tf.keras.models.Model(inputs=regression_input_layer, outputs=regression_output)

# Compile the combined model with the desired loss function and optimizer
combined_model.compile(loss='mse', optimizer='adam')

ValueError: Exception encountered when calling layer "regression_activation" (type Lambda).

Dimensions must be equal, but are 6 and 2 for '{{node regression_activation/mul}} = Mul[T=DT_FLOAT](Placeholder, Placeholder_1)' with input shapes: [?,6], [?,2].

Call arguments received by layer "regression_activation" (type Lambda):
  • inputs=['tf.Tensor(shape=(None, 6), dtype=float32)', 'tf.Tensor(shape=(None, 2), dtype=float32)']
  • mask=None
  • training=None

In [ ]:
import pandas as pd
dataset = pd.read_csv('D:\\NewFinalDatasetsForNeuralNetwork\\crash_noncrash_combined_csv_09_03_2023__13_24.csv')
dataset

In [ ]:
import seaborn as sns
sns.displot(dataset['TTC'], kde = True)

In [ ]:
#Splitting TTC from the other columns
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
from sklearn.model_selection import train_test_split

# 80-20 Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# This cell is only used for plotting the training dataset in the below cell, alongwith the column names
joined_train_dataset = X_train
train_dataset = pd.DataFrame(joined_train_dataset, columns = ['NORMALIZED PEDESTRIAN POSITION X', 'NORMALIZED PEDESTRIAN POSITION Y', 'NORMALIZED PEDESTRIAN DIRECTION', 'NORMALIZED PEDESTRIAN SPEED'])
train_dataset.loc[:,'TTC'] = y_train
#print(train_dataset)

In [ ]:
import tensorflow as tf
import keras
from keras import layers

# Defining first layer of the neural network, which normalizes the input data on the fly
normalizer_layer = keras.layers.Normalization(axis=-1)
normalizer_layer = layers.Normalization(input_shape=[4,], axis=None)
# Adapting normalizer layer to the input train data shape
normalizer_layer.adapt(np.array(X_train))

# Defines the model and compilation
def build_and_compile_model(normalizer_layer):
  nn_model = keras.Sequential([
      normalizer_layer,
      layers.Dense(64, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  nn_model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=['mse'])
  return nn_model

In [ ]:
# Building the neural network model and compiling it using the normalization layer adapted to the shape of our input training dataset
dnn_model = build_and_compile_model(normalizer_layer)
dnn_model.summary()

In [ ]:
# Report for training data for non linear model
#%%time
history = dnn_model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=64,
    verbose=1,
# Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
# ------------------------------------------------------------------------------------------------------------Regression Model----------------------------------------------------------------------------------------

In [ ]:
import numpy as np

# Create a new column 'TTC Category' and bin values based on their range
bins = [-1, 7, np.inf]
labels = ['crash', 'no_crash']
dataset['SCENARIO TYPE'] = pd.cut(dataset['TTC'], bins=bins, labels=labels)
dataset

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Select column to one hot encode
col_to_encode = 'SCENARIO TYPE'

# Perform one hot encoding
one_hot_encoded = pd.get_dummies(dataset[col_to_encode], prefix=col_to_encode)

# Append one hot encoded columns to original dataset
dataset = pd.concat([dataset, one_hot_encoded], axis=1)

# Drop original column that was one hot encoded
dataset = dataset.drop(col_to_encode, axis=1)
dataset

In [ ]:
# dataset['SCENARIO TYPE'] = data_transformed
dataset = dataset.drop('TTC', axis=1)
dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into features and labels
X = dataset.drop(columns=["SCENARIO TYPE_crash", "SCENARIO TYPE_no_crash"]).values
#X = dataset.drop(columns=["SCENARIO TYPE"]).values
#y = dataset[["SCENARIO TYPE"]].values
y = dataset[["SCENARIO TYPE_crash", "SCENARIO TYPE_no_crash"]].values
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
# Defining first layer of the neural network, which normalizes the input data on the fly
normalizer_layer = keras.layers.Normalization(axis=-1)
normalizer_layer = keras.layers.Normalization(input_shape=[4,], axis=None)
# Adapting normalizer layer to the input train data shape
normalizer_layer.adapt(np.array(X_train))

# Defines the model and compilation
def build_and_compile_model(normalizer_layer):
  nn_model = keras.Sequential([
      normalizer_layer,
      keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
      keras.layers.Dense(64, activation='sigmoid'),
      keras.layers.Dense(32, activation='relu'),
      keras.layers.Dense(y.shape[1], activation="softmax")
  ])

  nn_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=["categorical_accuracy"])
  return nn_model

In [ ]:
dnn_model = build_and_compile_model(normalizer_layer)
# Train the model
history = dnn_model.fit(X_train, y_train, epochs=300, batch_size=128, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = dnn_model.evaluate(X_test, y_test)
print("Test set accuracy:", accuracy)